In [1]:
import gym
from gym import error, spaces, utils
from gym.utils import seeding
import rclpy
from roboy_simulation_msgs.srv import GymStep
from roboy_simulation_msgs.msg import JointState
from roboy_simulation_msgs.srv import GymReset
import numpy as np

In [2]:
class MsjEnv(gym.GoalEnv):
    #metadata = {'render.modes': ['human']}
    def __init__(self):
        rclpy.init()
        self.node = rclpy.create_node('gym_client')
        self.step_cli = self.node.create_client(GymStep, 'gym_step')
        self.reset_cli = self.node.create_client(GymReset, 'gym_reset')
        #self.seed()
        #self.env_setup()


        #max limit position for q
        self.max_position = 3.14

        #2cm/s for ldot
        self.max_speed = 0.02
    
        #why do we need a sample goal -> desired goal for the training environment
        #sample goal is the q given
        #self.goal = self._sample_goal()

        #obs is a dictionary which has observation, achieved goal and desired goal. Achieved goal is useful for HER algorithm
        #obs = self._get_obs()

        #ldot is the action space. -2cm/s to 2cm/s
        self.action_space = spaces.Box(low=-self.max_speed, high=self.max_speed, shape=(8,), dtype='float32')
        
        #subscribe to the q and qdot. MSJ has 3 DOF q is a vector of 3. qdot has the same shape as q.
        #self.observation_space = spaces.Box(low=self.low_state, high=self.high_state)
        #enter the true joint limits
        """
        self.observation_space = spaces.Dict(dict(
            desired_goal=spaces.Box(-np.inf, np.inf, shape=obs['achieved_goal'].shape, dtype='float32'),
            achieved_goal=spaces.Box(-np.inf, np.inf, shape=obs['achieved_goal'].shape, dtype='float32'),
            observation=spaces.Box(-np.inf, np.inf, shape=obs['observation'].shape, dtype='float32'),
        ))
        """

    def __del__(self):
        self.node.destroy_node()
        rclpy.shutdown()
        
    # A function to initialize the random generator
    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]
    def env_setup():
        pass
        

    def step(self):
        #action = np.clip(action, self.action_space.low, self.action_space.high)
        #self._set_action(action) #set the topic on of target poses
        #self.sim.step() #unpause the simulation, implement a ROS2 service to command CARDSflow
        #obs = self._get_obs()
        
        #Cast action space to array given below
        set_points = [ 0.01,0.01,0.01,0.015,0.01,0.02,0.02,0.02]
        while not self.step_cli.wait_for_service(timeout_sec=1.0):
            self.node.get_logger().info('service not available, waiting...')

        step_size = 1.0
        req = GymStep.Request()
        res = GymStep.Response()
        req.set_points = set_points
        req.step_size = step_size
        future = self.step_cli.call_async(req)
        rclpy.spin_until_future_complete(self.node,future)
        if future.result() is not None:
            self.node.get_logger().info("result: %f" % future.result().q[1])
        return future.result()
    """
        done = False
        info = {
            'is_success': self._is_success(obs['achieved_goal'], self.goal),
        }
        reward = self.compute_reward(obs['achieved_goal'], self.goal, info)
        return obs, reward, done, info
    """
    def reset(self):
        #resetSim
        #unpauseSim
        #check topic publishers connection
        step_size = 1.0
        req = GymStep.Request()
        res = GymStep.Response()
        req.step_size = step_size
        future = self.reset_cli.call_async(req)
        rclpy.spin_until_future_complete(self.node,future)
        if future.result() is not None:
            self.node.get_logger().info("result: %f" % future.result().q[1])
        return future.result()
    

    def get_obs(self):
        #ROS topics and stuff 
        #position and velocity of the end effector
        #TO-DO subscribe to the q and qdot from ROS1_bridge
        while not self.step_cli.wait_for_service(timeout_sec=1.0):
            self.node.get_logger().info('service not available, waiting...')
        req = GymStep.Request()
        future = self.step_cli.call_async(req)
        rclpy.spin_until_future_complete(self.node,future)

        return future.result()
        """
        robot_qpos, robot_qvel = robot_get_obs(self.sim)
        achieved_goal = self._get_achieved_goal().ravel()
        observation = np.concatenate([robot_qpos, robot_qvel, achieved_goal])
        return {
            'observation': observation.copy(),
            'achieved_goal': achieved_goal.copy(),
            'desired_goal': self.goal.copy(),
        }
        """
    
    def _get_achieved_goal(self):
        
        #goal = [self.sim.data.get_site_xpos(name) for name in FINGERTIP_SITE_NAMES]
        return np.array(goal).flatten()
    
    def compute_reward(self, achieved_goal, goal, info):
        d = goal_distance(achieved_goal, goal)
        if self.reward_type == 'sparse':
            return -(d > self.distance_threshold).astype(np.float32)
        else:
            return -d



In [3]:
#rclpy.shutdown()
env = MsjEnv()



In [4]:
q = env.get_obs()
print(q)

roboy_simulation_msgs.srv.GymStep_Response(q=[0.14620553504372863, 0.005424070458055749, -0.13293526060327596], qdot=[0.0, 0.0, 0.0])


In [19]:
q = env.step()
print(q)

roboy_simulation_msgs.srv.GymStep_Response(q=[0.21515336087124917, 0.0974757112685054, -1.0690345169377136], qdot=[0.0, 0.0, 0.0])


In [6]:
q = env.reset()
print(q)

roboy_simulation_msgs.srv.GymReset_Response(q=[0.0, 0.0, 0.0], qdot=[0.0, 0.0, 0.0])


In [7]:
type(q.q)

list

In [8]:
q.qdot

[0.0, 0.0, 0.0]

In [9]:
class MsjROSBridgeProxy():

    def __init__(self):
        rclpy.init()
        self.node = rclpy.create_node('gym_client')
        self.step_cli = self.node.create_client(GymStep, 'gym_step')
        self.reset_cli = self.node.create_client(GymReset, 'gym_reset')
    
    def __del__(self):
        self.node.destroy_node()
        rclpy.shutdown()


    def forward_reset_command(self):
        step_size = 1.0
        req = GymStep.Request()
        res = GymStep.Response()
        req.step_size = step_size
        future = self.reset_cli.call_async(req)
        rclpy.spin_until_future_complete(self.node,future)
        if future.result() is not None:
            self.node.get_logger().info("result: %f" % future.result().q[1])
        q = future.result()
        return MsjRobotState(joint_angle=q.q, joint_vel=q.qdot)

In [10]:
def test_msj_ros_bridge_proxy():
    """reset function sets all joint angles to zero in simulation"""
    #assert "observation" in obs
    proxy = MsjROSBridgeProxy()
    new_robot_state = proxy.forward_reset_command()
    assert new_robot_state.joint_angle == [0,0,0]
    assert new_robot_state.joint_velocity == [0,0,0]

In [11]:
test_msj_ros_bridge_proxy()

RuntimeError: Failed to init: rcl_init called while already initialized, at /home/alex/ros2_install/src/ros2/rcl/rcl/src/rcl/rcl.c:75